In [5]:
import numpy as np
import pandas as pd
import lightgbm as lgb
import gc

print('Loading data ...')

train = pd.read_csv("C:\\Users\\Lin\\Desktop\\SMU\\Y3S1\\DM G1\\Project\\Codes\\data\\train.csv")
prop = pd.read_csv("C:\\Users\\Lin\\Desktop\\SMU\\Y3S1\\DM G1\\Project\\Codes\\data\\properties_2017.csv")

Loading data ...


C:\Users\Lin\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (49) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
train.head()

,parcelid,logerror,transactiondate,subdate
0,11016594,0.0276,1/1/2016,201601
1,14366692,-0.1684,1/1/2016,201601
2,12098116,-0.0040,1/1/2016,201601
3,12643413,0.0218,2/1/2016,201601
4,14432541,-0.0050,2/1/2016,201601


In [7]:
for c, dtype in zip(prop.columns, prop.dtypes):	
    if dtype == np.float64:		
        prop[c] = prop[c].astype(np.float32)

df_train = train.merge(prop, how='left', on='parcelid')

x_train = df_train.drop(['parcelid', 'logerror', 'transactiondate', 'propertyzoningdesc', 'propertycountylandusecode','subdate'], axis=1)
y_train = df_train['logerror'].values
print(x_train.shape, y_train.shape)

train_columns = x_train.columns

for c in x_train.dtypes[x_train.dtypes == object].index.values:
    x_train[c] = (x_train[c] == True)

del df_train; gc.collect()

split = 110000
x_train, y_train, x_valid, y_valid = x_train[:split], y_train[:split], x_train[split:], y_train[split:]
x_train = x_train.values.astype(np.float32, copy=False)
x_valid = x_valid.values.astype(np.float32, copy=False)

d_train = lgb.Dataset(x_train, label=y_train)
d_valid = lgb.Dataset(x_valid, label=y_valid)

params = {}
params['learning_rate'] = 0.002
params['boosting_type'] = 'gbdt'
params['objective'] = 'regression'
params['metric'] = 'mae'
params['sub_feature'] = 0.5
params['num_leaves'] = 60
params['min_data'] = 500
params['min_hessian'] = 1

watchlist = [d_valid]
clf = lgb.train(params, d_train, 500, watchlist)

del d_train, d_valid; gc.collect()
del x_train, x_valid; gc.collect()

print("Prepare for the prediction ...")
sample = pd.read_csv("C:\\Users\\Lin\\Desktop\\SMU\\Y3S1\\DM G1\\Project\\Codes\\data\\sample_submission.csv")
sample.columns = ['parcelid','201704','201705','201706','201707','201708','201709']
df_test = sample.merge(prop, on='parcelid', how='left')
del prop; gc.collect()
x_test = df_test[train_columns]
del df_test; gc.collect()
for c in x_test.dtypes[x_test.dtypes == object].index.values:
    x_test[c] = (x_test[c] == True)
x_test = x_test.values.astype(np.float32, copy=False)

print("Start prediction ...")
# num_threads > 1 will predict very slow in kernal
clf.reset_parameter({"num_threads":1})
p_test = clf.predict(x_test)

del x_test; gc.collect()

print("Start write result ...")

for c in sample.columns[sample.columns != 'parcelid']:
    sample[c] = p_test

sample.to_csv('lgb_bench.csv', index=False, float_format='%.4f')

(113039, 55) (113039,)
[1]	valid_0's l1: 0.0731009
[2]	valid_0's l1: 0.0730994
[3]	valid_0's l1: 0.0730981
[4]	valid_0's l1: 0.0730959
[5]	valid_0's l1: 0.0730933
[6]	valid_0's l1: 0.0730914
[7]	valid_0's l1: 0.0730886
[8]	valid_0's l1: 0.0730871
[9]	valid_0's l1: 0.0730854
[10]	valid_0's l1: 0.0730843
[11]	valid_0's l1: 0.0730825
[12]	valid_0's l1: 0.0730811
[13]	valid_0's l1: 0.0730799
[14]	valid_0's l1: 0.0730792
[15]	valid_0's l1: 0.0730774
[16]	valid_0's l1: 0.0730747
[17]	valid_0's l1: 0.0730727
[18]	valid_0's l1: 0.0730712
[19]	valid_0's l1: 0.0730701
[20]	valid_0's l1: 0.0730687
[21]	valid_0's l1: 0.0730674
[22]	valid_0's l1: 0.073065
[23]	valid_0's l1: 0.0730641
[24]	valid_0's l1: 0.0730621
[25]	valid_0's l1: 0.0730604
[26]	valid_0's l1: 0.0730586
[27]	valid_0's l1: 0.0730569
[28]	valid_0's l1: 0.0730549
[29]	valid_0's l1: 0.0730534
[30]	valid_0's l1: 0.0730517
[31]	valid_0's l1: 0.0730498
[32]	valid_0's l1: 0.0730483
[33]	valid_0's l1: 0.0730462
[34]	valid_0's l1: 0.0730443
[

[295]	valid_0's l1: 0.0729235
[296]	valid_0's l1: 0.0729244
[297]	valid_0's l1: 0.0729243
[298]	valid_0's l1: 0.0729251
[299]	valid_0's l1: 0.0729255
[300]	valid_0's l1: 0.0729252
[301]	valid_0's l1: 0.0729255
[302]	valid_0's l1: 0.0729261
[303]	valid_0's l1: 0.0729264
[304]	valid_0's l1: 0.0729265
[305]	valid_0's l1: 0.0729269
[306]	valid_0's l1: 0.0729273
[307]	valid_0's l1: 0.0729275
[308]	valid_0's l1: 0.072928
[309]	valid_0's l1: 0.0729286
[310]	valid_0's l1: 0.0729286
[311]	valid_0's l1: 0.0729287
[312]	valid_0's l1: 0.0729281
[313]	valid_0's l1: 0.0729288
[314]	valid_0's l1: 0.0729291
[315]	valid_0's l1: 0.0729291
[316]	valid_0's l1: 0.0729291
[317]	valid_0's l1: 0.0729294
[318]	valid_0's l1: 0.0729295
[319]	valid_0's l1: 0.0729297
[320]	valid_0's l1: 0.0729304
[321]	valid_0's l1: 0.0729295
[322]	valid_0's l1: 0.0729299
[323]	valid_0's l1: 0.0729302
[324]	valid_0's l1: 0.0729298
[325]	valid_0's l1: 0.0729305
[326]	valid_0's l1: 0.0729307
[327]	valid_0's l1: 0.0729313
[328]	valid